### Parte 2a: perfilamiento de datos

Se generó un reporte general sobre los datos. En lo que es relevante al contexto, interesa el número de registros vacías, datos más frecuentes, si es una variable nominal o numérica, si tiene alta cardinalidad, información estadística sobre la distribución de los datos, y número de registros únicos.

In [1]:
import os
import glob
import pandas as pd
import numpy as np
import pandas_profiling as pd_pf

In [4]:
df = pd.read_excel("./peajes2019.xlsx")
profile = pd_pf.ProfileReport(df)
profile.to_file("report.html")

### Generación de la tabla de hechos

Incluye la información del recaudo y tráfico por cada mes de los distintos años, acompañado del respectivo número, sigla y nombre del proyecto, y nombre de la estación peaje. **Corrección**: se removió las filas de peajes individuales que ya estaban expresadas como una sumatoria (quedaron peajes CS LA ESPERANZA, CS LAS FLORES, CS LOS GARZONES 1 Y LOS GARZONES 2). 

In [2]:
import pandas as pd
import numpy as np

In [ ]:
# Se lee la sheet v2 que contiene las correcciones mencionadas previamente
df1 = pd.read_excel("Recaudo y Trafico en Peajes 2016-2018.xlsx", sheet_name="Tabla Recaudo 2016-2018 v2")
df2 = pd.read_excel("Recaudo y Trafico en Peajes 2016-2018.xlsx", sheet_name="Tabla Trafico 2016-2018")

In [ ]:
df1.columns

In [57]:
def split_fechas(df_recaudo, df_trafico):
    x = []
    c = df_recaudo.columns

    for index, row in df_recaudo.iterrows():
        for j in range(len(row)):
            if j > 3:
                x.append([row[0], row[1], row[2], row[3], row[j]])
    i = 0
    for index2, row2 in df_trafico.iterrows():
        for j in range(len(row)):
            if j > 3:
                x[i].append(row2[j])
                x[i].append(c[j])
                i += 1
    df_f = pd.DataFrame(x, columns = ['No.', 'SiglaProyecto',
                                      'NombreProyecto', 'NombreEstacionPeaje',
                                      'Recaudo', 'Tráfico', 'Cod_fecha'])
    df_f.drop(columns=['No.', 'SiglaProyecto', 'NombreProyecto'], inplace=True)

    return df_f

### Generar archivo de Fechas

In [ ]:
x = []
meses = ['Enero','Febreo','Marzo','Abril','Mayo','Junio','Julio','Agosto','Septiembre','Octubre','Noviembre','Diciembre']
m = 0;
for i in range(4,40):
    cod_fecha = c[i]
    anio = cod_fecha[0:4]
    mes = meses[m]
    semestre = 1 if m<6 else 2
    cuad = 1 if m < 4 else (2 if m <8 else 3)
    tri = 1 if m < 3 else (2 if m <6 else (3 if m <9 else 4))
    x.append([cod_fecha,anio,semestre,cuad,tri,mes])
    m= (m+1)%12

fechas = pd.DataFrame(x, columns = ['Cod_fecha', 'Anio', 'Semestre', 'Cuadrimestre','Trimestre', 'Mes'])
fechas.tail()

In [ ]:
fechas.to_csv("Fechas.csv")

### Eliminar de Recaudo y trafico los peajes duplicados por falta de información específica

In [34]:
# Se lee la sheet v2 que contiene las correcciones mencionadas previamente
df1 = pd.read_excel("Recaudo y Trafico en Peajes 2016-2018.xlsx", sheet_name="Tabla Recaudo 2016-2018 v2")
df2 = pd.read_excel("Recaudo y Trafico en Peajes 2016-2018.xlsx", sheet_name="Tabla Trafico 2016-2018")

In [27]:
print(df1.shape, df2.shape)

(149, 40) (149, 40)


Primero con Recaudo, se eliminan las tildes, se ordenan y después se quitan las filas duplicadas

In [28]:
index = df1.index
q = len(index)
print('Cantidad inicial',q)

Cantidad inicial 149


In [36]:
cols = df1.select_dtypes(include=[np.object]).columns
df1[cols] = df1[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))

df1= df1.sort_values(by=['NombreEstacionPeaje'])

df1.drop_duplicates(subset='NombreEstacionPeaje', keep=False, inplace=True, ignore_index=True)


print('Cantidad final', df1.shape[0])

Cantidad final 113


In [21]:
print(df2.shape[0])

Int64Index([ 91,  70,  50, 138,  32, 115,  88,  62,   9, 101,
            ...
              5,   6,  81,  43,  68,  52,  82,  44, 125,  76],
           dtype='int64', length=113)


Ahora con trafico, se hace lo mismo: eliminan las tildes, se ordenan y después se quitan las filas duplicadas

In [ ]:
index = df2.index
q = len(index)
print('Cantidad inicial',q)

In [38]:
cols = df2.select_dtypes(include=[np.object]).columns
df2[cols] = df2[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))

df2= df2.sort_values(by=['NombreEstacionPeaje'])

df2.drop_duplicates(subset='NombreEstacionPeaje', keep=False, inplace=True, ignore_index=True)

print('Cantidad final',df2.shape[0])

Cantidad final 113


### Eliminar en el archivo peajes, el peaje duplicado
Ya que dejarlo implica incongruencias

In [39]:
peajes = pd.read_excel("peajes2019.xlsx", sheet_name="Peajes-2019")
cols = peajes.select_dtypes(include=[np.object]).columns
peajes[cols] = peajes[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))

peajes= peajes.sort_values(by=['nombre'])

peajes.drop_duplicates(subset='nombre', keep=False, inplace=True, ignore_index=True)

print('Cantidad final',peajes.shape[0])

Cantidad final 171


### Crear nuevas tablas sin las inconsistencias entre ambos archivos.

In [61]:
iterable_recaudo = df1['NombreEstacionPeaje']
iterable_peaje = peajes['nombre']

peajes = peajes.loc[peajes['nombre'].isin(iterable_recaudo)]

df1 = df1.loc[df1['NombreEstacionPeaje'].isin(iterable_peaje)]
df2 = df2.loc[df2['NombreEstacionPeaje'].isin(iterable_peaje)]

print('Dimensiones tabla peaje', peajes.shape)
print('Dimensiones tabla recaudo', df1.shape)
print('Dimensiones tabla tráfico', df2.shape)

df1.tail()

Dimensiones tabla peaje (90, 19)
Dimensiones tabla recaudo (90, 40)
Dimensiones tabla tráfico (90, 40)


,No.,SiglaProyecto,NombreProyecto,NombreEstacionPeaje,2016Ene,2016Feb,2016Mar,2016Abr,2016May,2016Jun,...,2018Mar,2018Abr,2018May,2018Jun,2018Jul,2018Ago,2018Sept,2018Oct,2018Nov,2018Dic
107,76,RC,RUTA CARIBE,TURBACO,1362975000,1298356600,1382575400,1316242400,1350494200,1306161800,...,1828402700,1775466750,1786483600,1723735650,1828610600,1849901100,1792488700,1840293000,1723673900,1957579900
108,54,BTS,BRICENO - TUNJA - SOGAMOSO,TUTA,3607602450,3028308850,3487426750,3072710950,3322622800,3009448600,...,3986007200,3722909750,3880672950,4010818850,4208486000,4247865100,4037116050,4337500050,4274456100,5158733200
109,90,RDS3,RUTA DEL SOL SECTOR 3,VALENCIA,873931600,730116800,811270000,770638100,785178000,789758000,...,894543600,930417350,877941550,881282450,930497200,886486800,839408700,902214900,894156150,1050933450
110,46,MVC,MALLA VIAL DEL VALLE DEL CAUCA Y CAUCA,VILLARICA,2937552600,2640579300,3028157900,2617425900,2807074300,2185010300,...,3628838700,3363717300,3318184200,3393585400,3716513300,3632112600,3472156700,3658232900,0,0
112,84,RDS2,RUTA DEL SOL SECTOR 2,ZAMBITO,2738295800,2218692000,2386408900,2320200600,2242007100,2359300700,...,0,0,0,0,0,0,0,0,0,0


## Crear tabla de hechos



In [62]:
tabla_hechos = split_fechas(df_recaudo=df1, df_trafico=df2)

tabla_hechos.to_csv("Tabla_hechos_recaudo_trafico.csv")
peajes.to_csv('peajes.csv')